In [ ]:
import pandas as pd
import redis
r = redis.StrictRedis(host='bdl1.eng.tau.ac.il', port=6379)
r.flushdb() # Delete all keys of currently selected database instance.
r.flushall() # Delete all keys of entire database. 

In [ ]:
import pymongo

myclient = pymongo.MongoClient()

#use database named "organisation"
mydb = myclient["stud22"]

#use collection named "testers"
mycol = mydb["website"]

#### Drop unnecessary collections

In [ ]:
mydb.website.drop()
mydb.website.job.drop()
mydb.list_collection_names() # print collections list

#### add_company operation

In [ ]:
def add_company(company):
    print(company)
    # check the keys names:
    if company.keys()[0] != 'company_name':
        print('Please set company_name')
    if company.keys()[1] != 'company_description':
        print('Please set company_description')
    if r.sismember('companies:names', company.values()[0]):
        print('Company already exist')
    else:
        r.sadd('companies:names', company.values()[0]) ## add the company name to redis sets
        mycol.insert_one(company) ## add the company to the collection

#### add_job operation

In [ ]:
def add_job(job, company_name):
    r.incr(company_name)  # increase the job id for company name
    job_id = r.get(company_name) # get the job_id
    r.set(company_name +':' + job_id, job['status']) # add the job status to redis string
    job["job_id"] = job_id # add the job_id to the job dict
    result = mycol.update_many({"company_name": company_name}, {"$push":{"job_list":job}}) # add the job title to the collection

#### add_application operation

In [ ]:
def add_application(candidate, job_id, company_name):
    print(job_id,company_name +':' + job_id)
    if not (r.sismember(candidate['email'],company_name +':' + job_id)) and (r.get(company_name +':' + job_id) == 'open'):
        r.sadd(candidate['email'],company_name +':' + job_id)
        print('yes')
        myquery = {'$and' : [{"company_name": company_name}, {"job_list.job_id" : job_id}]}
        mycol.update_many(myquery, {"$push":{"job_list.$.applications_list": candidate}})
        #mycol.update_many(myquery, {"$push":{"applications_list":"YOSI77"}}) # add the job title to the collection

In [ ]:
def update_job_status(company_name, job_id, new_status):
    r.set(company_name +':' + job_id, new_status) # update the job status to the redis string
    myquery = {'$and' : [{"company_name": company_name}, {"job_list.job_id" : job_id}]}
    mycol.update_one(myquery, {"$set": {"job_list.$.status": new_status},"$currentDate": {"job_list.$.lastModified": True}})
    #mycol.update_many(myquery, {"$push":{"applications_list":"YOSI77"}}) # add the job title to the collection

In [ ]:
def show_number_of_jobs(location, job_title):
    myquery = {'$and' : [{"job_list.location" : location}, {"job_list.job_title" : job_title}, {"job_list.status" : "open"}]}
    result = mycol.count(myquery)
    print("Number of open "+job_title+" positions in "+location+" is", result)

In [ ]:
def show_candidates(company_name, job_id):
    data = mycol.find_one({"company_name": company_name})['job_list']
    for x in data:
        if x['job_id'] == job_id:
            app_list = x['applications_list']
            requirements = x['requirements']
    col = ["Email", "number_of_matches"]
    df = pd.DataFrame(columns=col)
    for x in app_list:
        num_matches = len(set(x['skills']) & set(requirements))
        row = {"Email":x['email'],"number_of_matches":num_matches}
        df = df.append(row,ignore_index=True)
        df = df.sort_values(by=['number_of_matches'],ascending=False)
    print(df.Email)

In [ ]:
def count_jobs_by_date():
    aggregation_function = mycol.aggregate([
    {"$unwind":"$job_list"},
    {"$group": {"_id": "$job_list.publish_date", "open_jobs": {"$sum": 1}}}
      
    ])
    col = ["Open", "Close"]
    df = pd.DataFrame(columns=col, index=pd.date_range(start='1/1/2020', end='1/08/2020'))

    for i in aggregation_function:
        df.loc[i["_id"],"Open"] = i["open_jobs"]

    aggregation_function = mycol.aggregate([
    {"$unwind":"$job_list"},
    {"$match":{"job_list.status":"close"}},
    {"$group": {"_id": "$job_list.lastModified", "close_jobs": {"$sum": 1}}}
      
    ])

    for i in aggregation_function:
        df.loc[i["_id"],"Close"] = i["close_jobs"]
    
    df = df.fillna(0)
    df = df.assign(Open_jobs=df.Open.cumsum())
    df = df.assign(Close_jobs=df.Close.cumsum())
    df = df.drop(columns=["Open","Close"])
    print(df)

In [ ]:
company1 = {'company_name':'TAU', 'company_description':'University'}
job1 = {'job_title':'bi developers', 'location': 'Tel Aviv',
'requirements':['python','big data','mongodb'],
'status':'open','publish_date':'01-02-2020'}
application1 = {'candidate_name':'laura', 'email':'laura@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', "skills": ['python','sql'],"application_date":
'01-02-2020 15:00:00'}

In [ ]:
company2 = {'company_name':'ELBIT', 'company_description':'University'}
job2 = {'job_title':'mechanical engineering', 'location': 'Rishon Lezion',
'requirements':['python','big data','mongodb'],
'status':'open','publish_date':'01-02-2020'}
application2 = {'candidate_name':'yosi', 'email':'yosimzd2@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', "skills": ['python','sql','simulation'],"application_date":
'01-02-2020 16:00:00'}

In [ ]:
job3 = {'job_title':'mechanical engineering2', 'location': 'Rishon Lezion',
'requirements':['python','big data','mongodb'],
'status':'open','publish_date':'01-05-2020'}
add_job(job3.copy(), 'TAU')

In [ ]:
application3 = {'candidate_name':'yosi', 'email':'nurit@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', "skills": ['python','big data','mongodb'],"application_date":
'01-02-2020 16:00:00'}

In [ ]:
application4 = {'candidate_name':'nurit', 'email':'nura@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', "skills": ['python','big data'],"application_date":
'01-02-2020 16:00:00'}

### count_candidates_by_job (for learning)

In [ ]:
# test:
import time
import datetime
date_string = "01/12/2000"
date = datetime.datetime.strptime(date_string, "%d/%m/%Y")
time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

In [ ]:
# test:
date_string = "01/12/2000"
date = datetime.datetime.strptime(date_string, "%d/%m/%Y")#+datetime.timedelta(days=1)
date

In [ ]:
# test:
now = datetime.datetime.now()
last_30_days = now-datetime.timedelta(days=30)
print("last_30_days =", last_30_days)
time.mktime(now.timetuple())

### count_candidates_by_job function (not completed)

In [ ]:
# count_candidates_by_job():
import time
import datetime
date_string = "09/05/2021" # example date 
for i in range(50): # create for example 50 applications 
    date = datetime.datetime.strptime(date_string, "%d/%m/%Y") + datetime.timedelta(days=-i)
    time_step_val = time.mktime(date.timetuple())
    r.zadd('TAU:1', time_step_val, date)
    #print(time_step_val)
# find the last application during the last 30 days
now = datetime.datetime.now()
last_30_days = now-datetime.timedelta(days=30)
r.zrangebyscore('TAU:1', time.mktime(last_30_days.timetuple()), time.mktime(now.timetuple()))

In [ ]:
add_company(company1.copy())
add_company(company2.copy())

In [ ]:
add_job(job1.copy(), 'TAU')
add_job(job2.copy(), 'ELBIT')

In [ ]:
add_application(application1, "1", "TAU")
add_application(application2, "1", "ELBIT")
add_application(application3, "1", "TAU")
add_application(application4, "1", "TAU")

In [ ]:
location = "Rishon Lezion"
job_title = "mechanical engineering"
show_number_of_jobs(location, job_title)

In [ ]:
company_name = "TAU"
job_id="1"
show_candidates(company_name, job_id)

In [ ]:
update_job_status("TAU", '1', "close")